In [ ]:
import os
import tensorflow as tf
#import keras
from keras.layers import Add,Multiply,Softmax,Input,TimeDistributed,Dense,Average,GlobalAveragePooling1D,Concatenate,Lambda,RepeatVector, Conv2D,ConvLSTM2D, MaxPooling2D,BatchNormalization,Flatten,Reshape,UpSampling2D
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import pylab as pl
from IPython import display
from IPython.core.display import HTML
from IPython.core.display import display as html_width
html_width(HTML("<style>.container { width:90% !important; }</style>"))
import tensorflow_probability as tfp
import matplotlib.image as mpimg
from matplotlib.gridspec import GridSpec
import imageio
from tqdm import tqdm
import cv2
from tensorflow.keras.layers import Activation, BatchNormalization, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import psutil

import PIL.Image as Image

import gc
from tensorflow.keras import backend as k


physical_devices = tf.config.experimental.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]  # Adjust as needed
            )
    except RuntimeError as e:
        print(e)

C:\Users\kikop\AppData\Local\Temp\ipykernel_29604\701048593.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display as html_width


In [11]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 
print(tf.__version__)

['/device:CPU:0']
2.13.0


In [12]:
obs_min = 4
obs_max = 8
train_N = 49
train_p = np.random.permutation(49)
j = 0
x = 0
test_p = []
print(train_p)

train_p_copy = train_p.copy()

for i in train_p_copy:
    if i < 25 and j < 5:
        test_p.append(i)
        j += 1
        train_p = [item for item in train_p if item != i]  # Remove from train_p
    if i > 25 and x < 5:
        test_p.append(i)
        x += 1
        train_p = [item for item in train_p if item != i]  # Remove from train_p

print("Remaining train_p:", train_p)
print("test_p:", test_p)

[43 15 14 47 28 22  0 11 12 33 29 45 24 26 31  2 25 39  6 30 18 20 23 41
 13  4 27 38 37  7  9 48 36 40  1 17 21  5 42 19 35 16 34  8 46  3 44 32
 10]
Remaining train_p: [12, 45, 24, 26, 31, 2, 25, 39, 6, 30, 18, 20, 23, 41, 13, 4, 27, 38, 37, 7, 9, 48, 36, 40, 1, 17, 21, 5, 42, 19, 35, 16, 34, 8, 46, 3, 44, 32, 10]
test_p: [43, 15, 14, 47, 28, 22, 0, 11, 33, 29]


# Deep Modality Blending Networks

Try now with only 2 inputs (img and touch)

rndm coef like default

if more worried about the image using a coef for the image >0.2

For testing give 1 to img coef for only giving image

For easy test give coef of 0.5 for only two inputs

always add rnd samples from 0 zone and touch zone if necessary

plot the touch.txt for each data folder

use the weights from default

Add noise


coef > 2
noise
plot for only one sensor
reduce/increase more the weight for the touch sensor

In [14]:
######################################################################### GET TRAIN SAMPLES ############################################################################

def get_train_sample(coef = -1):
    n = np.random.randint(obs_min, obs_max) + 1
    d = train_p[np.random.randint(0, 39)]
    
    if coef == -1:
         coef = np.random.uniform(0.2, 1.0)
    
    img_coef = np.ones((1, 128)) * coef
    touch_coef = np.ones((1, 128)) * (1 - coef)
    
    observation = np.zeros((1, n, 128, 128, 4)) 
    observation_touch = np.zeros((1, n, 9)) 
    target_X = np.zeros((1, 1))
    target_Y = np.zeros((1, 128, 128, 6))
    #target_Y_touch = np.zeros((1, 16))
    target_Y_touch = np.zeros((1, 8))
    
    pose = np.loadtxt('Data3/%d/touch_data.txt' % d)
    time_len = pose.shape[0]
    times = np.linspace(0, 1, time_len)

    # Identify zero and nonzero rows
    zero_rows = np.where(np.all(pose == 0, axis=1))[0]
    nonzero_rows = np.where(np.any(pose != 0, axis=1))[0]
    
    if len(nonzero_rows) == 0 or len(zero_rows) == 0:
        raise ValueError("Dataset does not contain both zero and nonzero rows.")

    # Ensure at least one sample from each
    num_from_each = min(n // 2, len(nonzero_rows), len(zero_rows))
    selected_nonzero = np.random.choice(nonzero_rows, num_from_each, replace=False)
    selected_zero = np.random.choice(zero_rows, num_from_each, replace=False)

    remaining_samples = n - 2 * num_from_each
    if remaining_samples > 0:
        additional_samples = np.random.choice(time_len, remaining_samples, replace=False)
    else:
        additional_samples = np.array([], dtype=int)
    selected_indices = np.concatenate((selected_nonzero, selected_zero, additional_samples))
    np.random.shuffle(selected_indices)
    for i, idx in enumerate(selected_indices):
        if i < n -1:
            observation[0, i, :, :, 0] = np.ones((128, 128)) * times[idx]
            image = mpimg.imread('Data3/%d/frame_%d.jpeg' % (d, idx))
            image_resized = cv2.resize(image, (128, 128))
            observation[0, i, :, :, 1:] = image_resized / 255.

            observation_touch[0, i, 0] = times[idx]
            observation_touch[0, i, 1:9] = pose[idx] / 255

    target_X[0, 0] = times[selected_indices[-1]]
    t_img = mpimg.imread('Data3/%d/frame_%d.jpeg' % (d, selected_indices[-1]))
    t_img_resize = cv2.resize(t_img, (128, 128))
    target_Y[0, :, :, :3] = t_img_resize / 255.
    target_Y_touch[0, :8] = pose[selected_indices[-1]] / 255

    return [observation, observation_touch, target_X, img_coef, touch_coef], [target_Y, target_Y_touch], d, selected_indices[-1], coef



######################################################################### GET TEST SAMPLES ############################################################################


def get_valid_sample():
    n = np.random.randint(obs_min, obs_max) + 1
    d = train_p[np.random.randint(0, 9)]

    coef = 1
    
    img_coef = np.ones((1, 128)) * coef
    touch_coef = np.ones((1, 128)) * (1 - coef)
    
    observation = np.zeros((1, n, 128, 128, 4)) 
    observation_touch = np.zeros((1, n, 9)) 
    target_X = np.zeros((1, 1))
    target_Y = np.zeros((1, 128, 128, 6))
    #target_Y_touch = np.zeros((1, 16))
    target_Y_touch = np.zeros((1, 8))
    
    pose = np.loadtxt('Data3/%d/touch_data.txt' % d)
    time_len = pose.shape[0]
    times = np.linspace(0, 1, time_len)

    # Identify zero and nonzero rows
    zero_rows = np.where(np.all(pose == 0, axis=1))[0]
    nonzero_rows = np.where(np.any(pose != 0, axis=1))[0]
    
    if len(nonzero_rows) == 0 or len(zero_rows) == 0:
        raise ValueError("Dataset does not contain both zero and nonzero rows.")

    # Ensure at least one sample from each
    num_from_each = min(n // 2, len(nonzero_rows), len(zero_rows))
    selected_nonzero = np.random.choice(nonzero_rows, num_from_each, replace=False)
    selected_zero = np.random.choice(zero_rows, num_from_each, replace=False)

    remaining_samples = n - 2 * num_from_each
    if remaining_samples > 0:
        additional_samples = np.random.choice(time_len, remaining_samples, replace=False)
    else:
        additional_samples = np.array([], dtype=int)
    selected_indices = np.concatenate((selected_nonzero, selected_zero, additional_samples))
    np.random.shuffle(selected_indices)
    for i, idx in enumerate(selected_indices):
        if i < n -1:
            observation[0, i, :, :, 0] = np.ones((128, 128)) * times[idx]
            image = mpimg.imread('Data3/%d/frame_%d.jpeg' % (d, idx))
            image_resized = cv2.resize(image, (128, 128))
            observation[0, i, :, :, 1:] = image_resized / 255.

            observation_touch[0, i, 0] = times[idx]
            observation_touch[0, i, 1:9] = pose[idx] / 255

    target_X[0, 0] = times[selected_indices[-1]]
    t_img = mpimg.imread('Data3/%d/frame_%d.jpeg' % (d, selected_indices[-1]))
    t_img_resize = cv2.resize(t_img, (128, 128))
    target_Y[0, :, :, :3] = t_img_resize / 255.
    target_Y_touch[0, :8] = pose[selected_indices[-1]] / 255

    return [observation, observation_touch, target_X, img_coef, touch_coef], [target_Y, target_Y_touch], d, selected_indices[-1]

#inp, out, _, _ = get_train_sample()

In [15]:
@tf.keras.utils.register_keras_serializable()
def custom_loss(y_true, y_predicted):
    mean, log_sigma = tf.split(y_predicted, 2, axis=-1)
    y_true_value, temp =tf.split(y_true,2,axis=-1)
    #mean = tf.nn.relu(mean)
    sigma = tf.nn.softplus(log_sigma)
    dist = tfp.distributions.MultivariateNormalDiag(loc=mean, scale_diag=sigma)
    loss = -tf.reduce_mean(dist.log_prob(y_true_value))
    return loss


class MemoryCallback(tf.keras.callbacks.Callback):
    def __init__(self, coef, step):
        super().__init__()
        self.coef = coef 
        self.step = step 
        self.best_val_acc = float('inf')  # Initialize with a large value

    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()
        logs = logs or {}  # Ensure logs is not None
        process = psutil.Process(os.getpid())  
        memory_usage = process.memory_info().rss / (1024 * 1024)  

        train_acc = logs.get("touch_values_mae", "N/A")  # Replace 'accuracy' with the correct metric
        val_acc = logs.get("val_touch_values_mae", "N/A")  

        if val_acc != "N/A" and train_acc != "N/A":
            # Save the model only if the validation MAE improves
            if val_acc < self.best_val_acc:
                self.best_val_acc = val_acc  # Update the best validation accuracy
                if os.path.exists("Info_model.txt"):
                    os.remove("Info_model.txt")
                with open("Info_model.txt", "a") as f:
                    f.write(f"Step: {self.step} \n Coef = {self.coef} \n")
                self.model.save('my_model.keras', overwrite=True)


In [ ]:
image_layer = Input(shape=(None,128,128,4), name="image_observation") 
touch_layer = Input(shape=(None,9), name="touch_observation") 
#class_layer = Input(shape=(None,2), name="class_observation") 
target_X_layer = Input(shape=(1,), name = 'target_X')
img_coef_layer = Input(shape=(128,), name = 'image_coef')
touch_coef_layer = Input(shape=(128,), name = 'touch_coef')
#class_coef_layer = Input(shape=(128,), name = 'class_coef')

encoder_touch_sizes = [64,64,128,128,256]

touch_encoder = TimeDistributed(Dense(32, activation = 'relu'))(touch_layer)
for channel_size in encoder_touch_sizes:
    touch_encoder = TimeDistributed(Dense(channel_size, activation = 'relu'))(touch_encoder)

touch_representations = TimeDistributed(Dense(128, activation='relu'))(touch_encoder) #128
touch_representation = GlobalAveragePooling1D()(touch_representations) 

multiplied_touch = Multiply()([touch_representation,touch_coef_layer])

###################################################

encoder_img_sizes = [64,64,128,128,256]

image_encoder = TimeDistributed(Conv2D(32,(3,3),padding='same',activation='relu'))(image_layer)
image_encoder = TimeDistributed(MaxPooling2D((2,2)))(image_encoder)
for channel_size in encoder_img_sizes:
    image_encoder = TimeDistributed(Conv2D(channel_size,(3,3),padding='same',activation='relu'))(image_encoder)
    image_encoder = TimeDistributed(MaxPooling2D((2,2)))(image_encoder)

image_flatten = TimeDistributed(Flatten())(image_encoder)
img_representations = TimeDistributed(Dense(128, activation='relu'))(image_flatten)
img_representation = GlobalAveragePooling1D()(img_representations) 

multiplied_img = Multiply()([img_representation,img_coef_layer])

general_representation = Add()([multiplied_touch,multiplied_img])

merged_layer = Concatenate(axis=-1, name='merged')([general_representation,target_X_layer])

decoder_representation = Dense(1024, activation='relu') (merged_layer)

" =============== Image Decoder =============== "
decoder_img = Reshape([2,2,256])(decoder_representation)
decoder_img_sizes = [256,128,128,64,64,32]

for channel_size in decoder_img_sizes:
    decoder_img = Conv2D(channel_size, (3,3), padding='same', activation='relu')(decoder_img)
    decoder_img = UpSampling2D((2, 2))(decoder_img)

img_output = Conv2D(16, (3,3), padding='same', activation='relu')(decoder_img)
img_output = Conv2D(8, (3,3), padding='same', activation='relu')(img_output)
img_output = Conv2D(6, (3,3), padding='same', activation='sigmoid', name = "last_conv2d")(img_output)
" =============== Image Decoder =============== "

" =============== Touch Decoder =============== "
decoder_touch = Dense(512, activation='relu')(decoder_representation)
decoder_touch = Dense(216, activation='relu')(decoder_touch)
decoder_touch = Dense(128, activation='relu')(decoder_touch)
decoder_touch = Dense(32, activation='relu')(decoder_touch)
touch_output = Dense(8, activation='sigmoid', name="touch_values")(decoder_touch)

" =============== Touch Decoder =============== "

model = Model([image_layer, touch_layer, target_X_layer, img_coef_layer, touch_coef_layer],[img_output, touch_output])
latent_model = Model([image_layer, touch_layer, target_X_layer, img_coef_layer, touch_coef_layer],general_representation)
model.summary()
#plot_model(model)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_observation (InputLa  [(None, None, 128, 128, 4)   0         []                            
 yer)                        ]                                                                    
                                                                                                  
 time_distributed_7 (TimeDi  (None, None, 128, 128, 32)   1184      ['image_observation[0][0]']   
 stributed)                                                                                       
                                                                                                  
 time_distributed_8 (TimeDi  (None, None, 64, 64, 32)     0         ['time_distributed_7[0][0]']  
 stributed)                                                                                   

In [ ]:

model.compile(
    optimizer= Adam(lr=1e-4),
    loss={
        'last_conv2d': custom_loss,  
        'touch_values': 'mse',  
    },
    loss_weights={
        'last_conv2d': 1,  
        'touch_values': 0.01, 
    },
    metrics={  # Add accuracy tracking
        'touch_values': ['mae'],  # Mean Absolute Error for touch values
    }
)

In [ ]:
os.makedirs("Plots/Touch_Values", exist_ok=True)
os.makedirs("Plots/Loss", exist_ok=True)

loss_checkpoint = 1000
plot_checkpoint = 1000
validation_checkpoint = 1000
validation_error = 9999999
validation_step = -1
max_training_step = 1000000

dataset = ['image','touch']

float_formatter = "{:.4f}".format
np.set_printoptions(formatter={'float_kind': float_formatter})

real_output = []
predict_output = []
touch_value_idx = 2
loss_history = [] 

for step in range(max_training_step):
    inp, out, _, _, coef = get_train_sample()
    val_data, val_labels, _, _ = get_valid_sample()
    history = model.fit(inp, out, verbose=0, validation_data=(val_data, val_labels), callbacks=[MemoryCallback(coef, step)]) 
    loss_history.append(history.history['loss'][0])  

    if step % plot_checkpoint == 0:
        display.clear_output(wait=True)

        inp, out, _, _, _ = get_train_sample()
        
        # Predict touch values
        pred_touch = model.predict(inp)[1][0]
        real_touch = out[1][0] 

        # Store full sequences (instead of single index)
        real_output.append(real_touch)  
        predict_output.append(pred_touch)

        # Convert lists to NumPy arrays for easier plotting
        real_output_np = np.array(real_output)
        predict_output_np = np.array(predict_output)

        # Plot touch waveform for last epoch
        plt.figure(figsize=(10, 5))
        plt.plot(real_output_np[-1], label="True Touch Values", color='blue')  # Last epoch real values
        plt.plot(predict_output_np[-1], label="Predicted Touch Values", color='red', linestyle='--')  # Last epoch predictions

        plt.xlabel("Integrated Sensors")
        plt.ylabel("Touch Value")
        plt.title("Touch Sensor Predictions (Waveform)")
        plt.legend()

        # Save the plot
        touch_plot_path = f"Plots/Touch_Values/touch_plot_step_{step}.png"
        plt.savefig(touch_plot_path)
        plt.close()


        # Plot loss trend
        plt.figure(figsize=(8, 5))
        plt.plot(loss_history, label="Loss", color='green')
        plt.xlabel("Training Steps")
        plt.ylabel("Loss")
        plt.title("Training Loss Trend")
        plt.legend()

        # Save the loss plot
        loss_plot_path = f"Plots/Loss/loss_plot_step_{step}.png"
        plt.savefig(loss_plot_path)
        plt.close()

1/1 [==============================] - 0s 88ms/step


KeyboardInterrupt: 